In [ ]:
import import_ipynb
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
# Maybe I can Add a "90% of the centriole limit" or something like that
def moving_average_angle(a_centriole_list, a_lateral_size, lat_step, a_longitudinal_size = 0.2, long_step = 0.2):
    """ Function that calculating a moving average both in AnteroPosterior and MedioLateral Axis
    Arguments:
        -> a_centriole_list : a list of centrioles 
    """
    # Verification of 'perfect' lateral parameters
    n_lat_segment = int(  (1/lat_step) - (a_lateral_size/lat_step) +1)
    if  n_lat_segment != (1/lat_step-a_lateral_size/lat_step +1):
        print('Your parameters do not allow to cover the laterality of the worm equally (the last segment is not equal to all the other and will not be calculated)')
        print(n_lat_segment, 1/lat_step-a_lateral_size/lat_step)
    
    # Verification of 'perfect' longitudinal parameters  
    n_long_segment = int(1/long_step-a_longitudinal_size/long_step) + 1
    if  n_long_segment != (1/long_step-a_longitudinal_size/long_step + 1):
        print('Your parameters do not allow to cover the length of the worm equally (the last segment is not equal to all the other and will not be calculated)')
        print(n_long_segment, 1/long_step-a_longitudinal_size/long_step)

    # Table instantiation
    counter = np.zeros((n_long_segment, n_lat_segment))
    cos_sum = np.zeros((n_long_segment, n_lat_segment))
    sin_sum = np.zeros((n_long_segment, n_lat_segment))
    
    # Iterate thourh each centriole of the list
    for a_centriole in a_centriole_list:
        # Recalculate the lateral distance (left = 0, right = 1)
        if a_centriole[5] == 'left':
            lat_ctrl = abs(a_centriole[6]-1)
        else: 
             lat_ctrl = a_centriole[6] + 1
        
        lat_ctrl = lat_ctrl/2
        lon_ctrl = a_centriole[7]
        
        
        # Does the centriole belong to this longitudinal segment?
        for long_ in range(n_long_segment):
            if (0 + long_step*long_) <= lon_ctrl <= (a_longitudinal_size+long_step*long_):
                # if YES, Does the centriole belong to this longitudinal segment?
                for lat_ in range(n_lat_segment):
                    if (0 + lat_step*lat_) <= lat_ctrl <= (a_lateral_size+lat_step*lat_):
                        
                        # if YES, modifiy, counter and sum cos and sin in appropriate position
                        counter[long_][lat_] += 1
                        cos_sum[long_][lat_] += math.cos(math.radians(a_centriole[8]))
                        sin_sum[long_][lat_] += math.sin(math.radians(a_centriole[8]))
    
    
    print(f'Analysis computed on {n_long_segment} Antero_posterior segment and {n_lat_segment} Medio-lateral segment')
    
    # if count = 0 => Divsion by 0
    # replace all the count = 0 by count = 1 to be able to compute the angle average
    # since if count = 0, angle = 0 , the result is not affected BUT final table is false (0 should be replace by NAN)
    counter = np.reshape([np.nan if ele == 0 else ele for ele in  counter.flatten()], (n_long_segment, n_lat_segment))
    
    cos_mean, sin_mean = cos_sum/counter , sin_sum/counter
    cos_std = np.std(cos_sum), np.std(sin_sum)
    
    angle_R = ((cos_sum/counter)**2+(sin_sum/counter)**2)**0.5
    angle_std = np.degrees((-2*np.log(angle_R))**2)
    #angle_dispersion = (1-angle_R)/(2*angle_R**2)
    
    angle_mean = np.degrees(np.arctan2(sin_mean, cos_mean))
    
    
    return angle_mean, angle_std